In [1]:
import pandas as pd 
import numpy as np
from pathlib import Path
from shapely.geometry import Polygon, Point
from shapely.ops import transform
from functools import partial
import pyproj
from classLib_forFlightplans import *
import random
import operator
from functools import reduce

In [2]:
region = [100,0,110,0,110,10,100,10]
Path.cwd()
filteraws = pd.read_json("../airways.json")

"""This snippet adds transformed x and y columns to the airways file"""
a=np.array([0,100])
a_ = np.array([a[1],a[0]])
b=  np.array([0,110])
b_ = np.array([b[1],b[0]])
c =np.array([10,110])
c_ = np.array([c[1],c[0]])
d=  np.array([10,100])
d_ = np.array([d[1],d[0]])

horizontal_len= b[1]-a[1]
vertical_len = d[0]-a[0]

dict1=[]

for i in range(len(filteraws)):
    # print(i)
    d1= []
    for j in range(len(filteraws.segments[i])):
        # print(j)
        P = np.array([float(filteraws.segments[i][j]['lon']),float(filteraws.segments[i][j]['lat'])])
        # print(P)
        # print("^^")
        # vert_d = np.linalg.norm(np.cross(b_-a_,P-a_))/np.linalg.norm(b_-a_)
        # horiz_d = np.linalg.norm(np.cross(d_-a_,P-a_))/np.linalg.norm(d_-a_)
        vert_d = P[1]-a_[1]
        horiz_d = P[0] -a_[0]
        airdict1={
                # "name":filteraws.name[i],        
                # "segments":[{"name":filteraws.segments[i][j]['name'],
                "name":filteraws.segments[i][j]['name'],
                "lat":filteraws.segments[i][j]['lat'],
                "lon":filteraws.segments[i][j]['lon'],
                "latdms":filteraws.segments[i][j]['latdms'],
                "londms":filteraws.segments[i][j]['londms'],
                "transf_x": (900/horizontal_len)*horiz_d,
                "transf_y": (900/vertical_len)*vert_d
                }
        d1.append(airdict1)
    dict3 ={"name":filteraws.name[i], 
            "segments":d1}
    dict1.append(dict3)


In [4]:
datalist=[]
for scenarios in range(10):
    df = pd.DataFrame(dict1)
    waypointList = []
    for i in range(len(df)):
        for j in range(len(df.segments[i])):
            waypointList.append(WayPoint(df.segments[i][j]['name'],[df.segments[i][j]['transf_x'],900-df.segments[i][j]['transf_y']]))


    route1 = ['MELAS (VVTS/WSJC FIR BDRY)','MABLI','MUMSO','MABAL','KILOT','KIBOL WSJC/WMFC FIR BDRY','PEKLA']
    route2 = ['ENREP','NOPAT','DAMOG','SUSAR','MUMSO','MABAL','KILOT','KIBOL WSJC/WMFC FIR BDRY','PEKLA']

    # r_set1=set(route1)
    # r_set2=set(route2)
    # r2itemsnotinr1 = list(r_set2-r_set1)
    # uniquepoints= route1 +r2itemsnotinr1
    # print(uniquepoints)
    # print(len(uniquepoints))
    r1=[]
    for i in (route1):
        for j in range(len(waypointList)):
            if i == waypointList[j].name:
                r1.append(waypointList[j])
                break
    r2=[]
    for i in (route2):
        for j in range(len(waypointList)):
            if i == waypointList[j].name:
                r2.append(waypointList[j])
                break


    allroutes = r1 + r2
    print(len(allroutes))
    airwayList=[]
    for i in range(len(allroutes)-1):
        aw =[allroutes[i],allroutes[i+1]]
        airwayList.append(Airways('aw'+str(i),aw[0],aw[1]))
    print(len(airwayList))
    #to add more airways i can edit and append waypoints here.

    aircraft_type=['medium','heavy','super_heavy']
    aircraft_paths=[r1,r2]
    aircraft_paths_x = []
    aircraft_paths_y = []
    path_waypoints   = []
    for i in range(len(aircraft_paths)):
        x_list=[]
        y_list=[]
        wps_list=[]
        for j in range(len((aircraft_paths[i]))):
            wps_list.append(aircraft_paths[i][j].name)
            x=aircraft_paths[i][j].x
            y=aircraft_paths[i][j].y
            x_list.append(x)
            y_list.append(y)
        aircraft_paths_x.append(x_list)
        aircraft_paths_y.append(y_list)
        path_waypoints.append(wps_list)

    ac_per_scenario = 2
    aircraftList=[]
    aircraft_name=[]
    for n in range(1,ac_per_scenario+1):
        aircraft_name.append('ac'+str(n))
    for i in range(ac_per_scenario):
        route = np.random.choice(aircraft_paths)
        # print(point)
        d=[20,30,40,50,60,70,80,90,100,120,130] #offset,
        aircraft = Aircraft(str(aircraft_name[i]), route[0] ,route[1], route,  np.random.choice(d)) #this will create multiple scenarios
        """here route[1] instead of route[-1] because the offset is calculate based on the end point of the route."""
        aircraftList.append(aircraft)


    x_dirlist=[]
    y_dirlist=[]
    for i in range(len(aircraftList)):
        x_dir =[]
        y_dir =[]
        for j in range(len(aircraftList[i].route)-1):
            startpoint = np.array([aircraftList[i].route[j].x, aircraftList[i].route[j].y])
            endpoint = np.array([aircraftList[i].route[j+1].x, aircraftList[i].route[j+1].y])
            segment = np.linalg.norm(startpoint - endpoint)
            xdir = (endpoint[0] - startpoint[0])/segment
            ydir = (endpoint[1] - startpoint[1])/segment
            x_dir.append(xdir)
            y_dir.append(ydir)
        x_dirlist.append(x_dir)
        y_dirlist.append(y_dir)

    vel=0.25
    timelist=[]
    x_loclist=[]
    y_loclist=[]
    total_time=[]
    for i in range(len(aircraftList)):
        x_loc=[]
        y_loc=[]
        time=[]
        section_time=[]
        for j in range(len(aircraftList[i].route)-1):
            startpoint = np.array([aircraftList[i].route[j].x, aircraftList[i].route[j].y])
            endpoint = np.array([aircraftList[i].route[j+1].x, aircraftList[i].route[j+1].y])
            distance = np.linalg.norm(startpoint - endpoint)
            totaltime = distance/vel    #(time per kilometer in seconds, bwtween two waypoints)
            
            for t in range(int(totaltime)):
                x_point = startpoint[0] + t*x_dirlist[i][j]*vel
                y_point = startpoint[1] + t*y_dirlist[i][j]*vel
                x_loc.append(x_point)
                y_loc.append(y_point)
                time.append(t)
                
            section_time.append(totaltime)
        # print(len(section_time))
        timelist.append(time)
        x_loclist.append(x_loc)
        y_loclist.append(y_loc)
        total_time.append(section_time)

    all_dirs_x=[]
    all_dirs_y=[]
    for i in range(len(total_time)):
        x_list=[]
        y_list=[]
        for j in range(len(total_time[i])):
            x = np.repeat(x_dirlist[i][j], int(total_time[i][j]))
            y = np.repeat(y_dirlist[i], int(total_time[i][j]))
            print(x)
            x= x.tolist()
            y= y.tolist()
            # print(len(x))
            # print(len(y[]))
            x_list.append(x)
            y_list.append(y)
        
        all_dirs_x.append(x_list)
        all_dirs_y.append(y_list)


    x_directions_flat=[]
    y_directions_flat=[]
    for i in range(len(x_dirlist)):
        x_l= all_dirs_x[i]
        x_flat = reduce(operator.concat,x_l)
        x_directions_flat.append(x_flat)

        y_l = all_dirs_y[i]
        y_flat = reduce(operator.concat,y_l)
        y_directions_flat.append(y_flat)

    dict4={}
    for i in range(len(waypointList)):
        waydict={waypointList[i].name:{
            'x':waypointList[i].x,
            'y':waypointList[i].y}}
        dict4.update(waydict)


    dict5={}
    for i in range(len(airwayList)):
        airwaydict= {airwayList[i].name:{
            'start':airwayList[i].start_wp,
            'end' : airwayList[i].end_wp}}
        dict5.update(airwaydict)


    dict6={}
    for i in range(len(aircraftList)):
        aircraftdict= {
            aircraftList[i].name:{
            'x': aircraftList[i].offset_x,
            'y':aircraftList[i].offset_y,
            'dir_x': x_dirlist[i][0],
            'dir_y': y_dirlist[i][0],
            'x_dirs':x_directions_flat[i],
            'y_dirs':y_directions_flat[i],
            'type': np.random.choice(aircraft_type),
            'altitude':360,
            'path_waypoints': path_waypoints[i],
            'x_path' : x_loclist[i],
            'y_path' : y_loclist[i],
            'timestep': timelist[i]
            }
            }
        dict6.update(aircraftdict)


    df2=pd.read_json('SgSectors_LonLat.json')


    """the sqaure region with 'a' as the bottom left point (moving anticlock abcd)
        a,b,c,d= [lon,lat] format"""
    a = np.array([100,0])
    b = np.array([110,0])
    c = np.array([110,10])
    d = np.array([100,10])

    horizontal_length  = b[0]-a[0]
    vertical_length = d[1]-a[1]
    keylist=['sector1','sector2','sector3','sector4','sector5','sector6','sector7','sector8']
    l1=[[],[],[],[],[],[],[],[]]
    for i in range(len(keylist)):
        for j in range(len(df2.Sectors[0][keylist[i]])):
            P= np.array([df2.Sectors[0][keylist[i]][j][0],df2.Sectors[0][keylist[i]][j][1]])
            # print(P)
            vert_d = P[1]-a[1]
            horiz_d= P[0]-a[0]
            l1[i].append([(900/horizontal_length)*horiz_d,900-(900/vertical_length)*vert_d])



    data_dict={'waypoints':dict4, 'airways':dict5,'aircrafts':dict6, 
            'sectors':{
                        'sector1':{'type': 'secondary', 'boundary': l1[0]},
                        'sector2':{'type': 'secondary', 'boundary': l1[1]},
                        'sector3':{'type': 'secondary', 'boundary': l1[2]},
                        'sector4':{'type': 'secondary', 'boundary': l1[3]},
                        'sector5':{'type': 'secondary',  'boundary': l1[4]},
                        'sector6':{'type': 'primary', 'boundary':  l1[5]},
                        'sector7':{'type':'secondary', 'boundary': l1[6]},
                        'sector8':{'type':'secondary' ,'boundary':l1[7]}
                        }}

    datalist.append(data_dict)

import json
with open('scenario_data/26August_custom_flightplanned_Scenarios3.json', 'w') as json_file:
    json.dump(datalist, json_file)

6 -0.77633506 -0.77633506 -0.77633506
 -0.77633506 -0.77633506 -0.77633506 -0.77633506 -0.77633506 -0.77633506
 -0.77633506 -0.77633506 -0.77633506 -0.77633506 -0.77633506 -0.77633506
 -0.77633506 -0.77633506 -0.77633506 -0.77633506 -0.77633506 -0.77633506
 -0.77633506 -0.77633506 -0.77633506 -0.77633506 -0.77633506 -0.77633506
 -0.77633506 -0.77633506 -0.77633506 -0.77633506 -0.77633506 -0.77633506
 -0.77633506 -0.77633506 -0.77633506 -0.77633506 -0.77633506 -0.77633506
 -0.77633506 -0.77633506 -0.77633506 -0.77633506 -0.77633506 -0.77633506
 -0.77633506 -0.77633506 -0.77633506 -0.77633506 -0.77633506 -0.77633506
 -0.77633506 -0.77633506 -0.77633506 -0.77633506 -0.77633506 -0.77633506
 -0.77633506 -0.77633506 -0.77633506 -0.77633506 -0.77633506 -0.77633506
 -0.77633506 -0.77633506 -0.77633506 -0.77633506 -0.77633506 -0.77633506
 -0.77633506 -0.77633506 -0.77633506 -0.77633506 -0.77633506 -0.77633506
 -0.77633506 -0.77633506 -0.77633506 -0.77633506 -0.77633506 -0.77633506
 -0.77633506 

16
15


In [4]:
# len(aircraft_paths_x[1]),len(airwayList)
np.random.choice(aircraft_paths)

In [5]:
aircraftList[0].route